In [2]:
import pandas as pd

# 读取用户上传的 CSV 文件
file_path_1 = '附件1处理后.csv'
file_path_2 = '附件2处理后.csv'

try:
    df_inventory = pd.read_csv(file_path_1, encoding='utf-8')
    df_sales = pd.read_csv(file_path_2, encoding='utf-8')
except UnicodeDecodeError:
    # 若 utf-8 编码失败，尝试使用 gbk 编码读取
    df_inventory = pd.read_csv(file_path_1, encoding='gbk')
    df_sales = pd.read_csv(file_path_2, encoding='gbk')

# 查看前几行数据以了解数据的基本结构
df_inventory_head = df_inventory.head()
df_sales_head = df_sales.head()

# 显示数据表头信息，以便用户确认数据结构
df_inventory_head, df_sales_head


(            品类       月份      库存量
 0   category84  2022年7月  3689222
 1  category225  2022年7月  3095300
 2   category21  2022年7月  2379868
 3   category42  2022年7月  2306150
 4  category209  2022年7月  1864077,
             品类        日期     销量
 0   category21  2022/7/1  31424
 1   category84  2022/7/1  29874
 2  category254  2022/7/1  22190
 3  category308  2022/7/1  11726
 4   category42  2022/7/1   9978)

In [3]:
# 查找库存量数据和销量数据中的缺失值情况
inventory_missing = df_inventory.isnull().sum()
sales_missing = df_sales.isnull().sum()

# 显示缺失值情况
inventory_missing, sales_missing


(品类     0
 月份     0
 库存量    0
 dtype: int64,
 品类    0
 日期    0
 销量    0
 dtype: int64)

In [ ]:
import matplotlib.pyplot as plt
import random
# 设置matplotlib绘图时可以显示中文
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置字体为黑体
plt.rcParams['axes.unicode_minus'] = False  # 使得坐标轴负号显示正常
# 从库存量和销量数据中随机选取3个品类
categories = df_inventory['品类'].unique()
selected_categories = random.sample(list(categories), 3)

# 过滤出选取品类的库存量和销量数据
selected_inventory = df_inventory[df_inventory['品类'].isin(selected_categories)].copy()
selected_sales = df_sales[df_sales['品类'].isin(selected_categories)].copy()

# 转换月份和日期为 pandas 日期格式，方便按时间绘图
selected_inventory.loc[:, '月份'] = pd.to_datetime(selected_inventory['月份'], format='%Y年%m月')
selected_sales.loc[:, '日期'] = pd.to_datetime(selected_sales['日期'], format='%Y/%m/%d')

# 选择颜色，为每个品类指定不同的颜色
color_mapping = {
    selected_categories[0]: 'red',      # 自定义颜色，可以改为 'blue', 'green', 等
    selected_categories[1]: 'blue',
    selected_categories[2]: 'green'
}

# 绘制库存量随时间变化图
plt.figure(figsize=(12, 6))
for category in selected_categories:
    inventory_data = selected_inventory[selected_inventory['品类'] == category]
    plt.plot(inventory_data['月份'], inventory_data['库存量'], label=f'{category} 库存量', color=color_mapping[category])
plt.xlabel('时间')
plt.ylabel('库存量')
plt.title('随机选取的3个品类库存量随时间变化图')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:

# 绘制销量随时间变化图
plt.figure(figsize=(12, 6))
for category in selected_categories:
    sales_data = selected_sales[selected_sales['品类'] == category]
    plt.plot(sales_data['日期'], sales_data['销量'], label=f'{category} 销量', color=color_mapping[category])
plt.xlabel('时间')
plt.ylabel('销量')
plt.title('随机选取的3个品类销量随时间变化图')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import numpy as np

# 读取库存量数据
inventory_file_path = "附件1处理后.csv"  # 请确保文件路径正确
df_inventory = pd.read_csv(inventory_file_path, encoding='gbk')

# 转换月份列为 pandas 日期类型
df_inventory['月份'] = pd.to_datetime(df_inventory['月份'], format='%Y年%m月')

# 提取2023年1-6月的数据和2022年7-9月的数据
inventory_2023 = df_inventory[(df_inventory['月份'] >= '2023-01-01') & (df_inventory['月份'] <= '2023-06-01')]
inventory_2022 = df_inventory[(df_inventory['月份'] >= '2022-07-01') & (df_inventory['月份'] <= '2022-09-01')]

# 初始化一个字典来存储每个品类的预测结果
inventory_predictions = {}

# 获取所有品类
all_categories = df_inventory['品类'].unique()

# 对每个品类进行灰色预测 (GM(1,1))
for category in all_categories:
    # 获取该品类的今年和去年数据
    data_2023 = inventory_2023[inventory_2023['品类'] == category]['库存量'].values
    data_2022 = inventory_2022[inventory_2022['品类'] == category]['库存量'].values

    # 如果数据不足，跳过该品类
    if len(data_2023) < 1 or len(data_2022) < 1:
        continue

    # 确保两段数据长度相同，以便进行融合
    min_length = min(len(data_2023), len(data_2022))
    data_2023 = data_2023[:min_length]
    data_2022 = data_2022[:min_length]

    # 加权融合数据
    fused_data = 0.7 * data_2023 + 0.3 * data_2022

    # 将融合后的数据作为灰色预测的输入序列
    original_sequence = fused_data

    # 实现灰色预测（GM(1,1)）
    def GM11(x0):
        # 1. 累加生成序列
        x1 = np.cumsum(x0)

        # 2. 构造数据矩阵和Y向量
        B = np.vstack([-0.5 * (x1[:-1] + x1[1:]), np.ones(len(x1) - 1)]).T
        Y = x0[1:]

        # 3. 使用最小二乘法估计参数a和b
        a, b = np.linalg.lstsq(B, Y, rcond=None)[0]

        # 4. 预测公式 x(t) = (x0(1) - b/a) * exp(-a * (t-1)) + b/a
        def predict(t):
            return (x0[0] - b / a) * np.exp(-a * (t - 1)) + b / a

        return predict

    # 使用GM(1,1)模型进行预测
    gm_model = GM11(original_sequence)

    # 预测未来3个月的库存量（7月、8月、9月）
    predicted_inventory = [gm_model(i) for i in range(len(original_sequence) + 1, len(original_sequence) + 4)]

    # 保存预测结果
    inventory_predictions[category] = predicted_inventory


    
# 转换预测结果为 DataFrame
inventory_predictions_df = pd.DataFrame(inventory_predictions, index=['2023年7月', '2023年8月', '2023年9月'])

# 保存预测结果到文件
inventory_predictions_file_path = "库存量预测结果.csv"
inventory_predictions_df.to_csv(inventory_predictions_file_path, index=True)

print(f"库存量预测完成，结果已保存至 {inventory_predictions_file_path}")


库存量预测完成，结果已保存至 库存量预测结果.csv


C:\Users\28578\AppData\Local\Temp\ipykernel_37612\2724441320.py:56: RuntimeWarning: divide by zero encountered in scalar divide
  return (x0[0] - b / a) * np.exp(-a * (t - 1)) + b / a
C:\Users\28578\AppData\Local\Temp\ipykernel_37612\2724441320.py:56: RuntimeWarning: invalid value encountered in scalar add
  return (x0[0] - b / a) * np.exp(-a * (t - 1)) + b / a


In [ ]:
# 提取销量特征的完整代码
import pandas as pd
import numpy as np
from datetime import timedelta
import random

# 假设销量数据文件为 "sales_data.csv"，包含列 ["品类", "日期", "销量"]
file_path = "附件2处理后.csv"

# 读取销量数据
# 由于文件中可能包含中文列名，使用 utf-8 编码读取
try:
    df_sales = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df_sales = pd.read_csv(file_path, encoding='gbk')

# 将日期列转换为 pandas 日期类型
df_sales['日期'] = pd.to_datetime(df_sales['日期'], format='%Y/%m/%d')

# 基于已知的 2022 年 7 月 1 日是星期五，来计算所有日期的星期几（星期几从 1 开始）
known_date = pd.Timestamp('2022-07-01')
known_weekday = 5  # 1 表示周一，5 表示周五

# 重新计算每个日期的星期几，范围从 1（周一）到 7（周日）
df_sales['星期几'] = df_sales['日期'].apply(lambda date: ((known_weekday - 1) + (date - known_date).days) % 7 + 1)

# 计算是否周末（6 和 7 表示周六和周日）
df_sales['是否周末'] = df_sales['星期几'].apply(lambda x: 1 if x in [6, 7] else 0)

# 手动加入一些常见的法定节假日标注，仅标注现有数据范围内的节假日
holidays_manual = [
    pd.Timestamp('2022-07-01'),  # 建党节
    pd.Timestamp('2023-04-05'),  # 清明节
    pd.Timestamp('2023-05-01'),  # 劳动节
    pd.Timestamp('2023-06-22'),  # 端午节
    pd.Timestamp('2022-09-10'),  # 中秋节
    pd.Timestamp('2023-09-29'),  # 中秋节
]
df_sales['是否节假日'] = df_sales['日期'].apply(lambda date: 1 if date in holidays_manual else 0)

# 手动加入一些常见的促销日，仅标注现有数据范围内
promotions = [
    pd.Timestamp('2022-06-18'), pd.Timestamp('2023-06-18'),  # 618 购物节
    pd.Timestamp('2022-09-09'), pd.Timestamp('2023-09-09'),  # 99 大促
]
df_sales['是否促销日'] = df_sales['日期'].apply(lambda date: 1 if date in promotions else 0)

# 为所有品类构造滞后特征（例如，使用前 7 天的销量来预测当前销量）
for lag in range(1, 8):
    df_sales[f'滞后_{lag}天'] = df_sales.groupby('品类')['销量'].shift(lag)

# 去除由于构造滞后特征而导致缺失值的行
df_sales = df_sales.dropna()

# 准备最终包含所有特征的特征表格
feature_columns = ['日期', '品类', '销量', '月', '日', '星期几', '是否周末', '是否节假日', '是否促销日'] + [f'滞后_{lag}天' for lag in range(1, 8)]

# 提取月份和日期特征
df_sales['月'] = df_sales['日期'].dt.month
df_sales['日'] = df_sales['日期'].dt.day

# 最终特征数据表格
feature_df = df_sales[feature_columns]

# 保存特征表格到文件
feature_file_path = "销量预测特征表.csv"
feature_df.to_csv(feature_file_path, index=False)

print(f"特征提取完成，已保存至 {feature_file_path}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 读取特征表格数据
feature_file_path = "销量预测特征表.csv"  # 请确保文件路径正确
df_features = pd.read_csv(feature_file_path)

# 挑选其中两个品类进行训练和评估
selected_categories = df_features['品类'].unique()[:2]
results = {}

for category in selected_categories:
    # 提取当前品类的数据
    df_category = df_features[df_features['品类'] == category]

    # 准备训练数据和标签
    feature_columns = ['月', '日', '星期几', '是否周末', '是否节假日', '是否促销日'] + [f'滞后_{lag}天' for lag in range(1, 8)]
    X = df_category[feature_columns]
    y = df_category['销量']

    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 初始化随机森林模型
    rf_model = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=42)

    # 训练模型
    rf_model.fit(X_train, y_train)

    # 使用测试集进行预测
    y_pred = rf_model.predict(X_test)

    # 评估模型表现
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    # 保存评估结果
    results[category] = {
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape
    }

# 输出两个品类的评估结果
print(results)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 读取特征表格数据
feature_file_path = "销量预测特征表.csv"  # 请确保文件路径正确
df_features = pd.read_csv(feature_file_path)

# 获取所有品类
all_categories = df_features['品类'].unique()

# 初始化一个字典来存储每个品类的评估结果
results = []

# 遍历每个品类，训练模型并进行评估
for category in all_categories:
    # 提取当前品类的数据
    df_category = df_features[df_features['品类'] == category]

    # 检查样本数量是否足够进行训练和测试划分
    if len(df_category) < 5:
        print(f"品类 {category} 样本数量不足，跳过训练和评估。")
        continue

    # 准备训练数据和标签
    feature_columns = ['月', '日', '星期几', '是否周末', '是否节假日', '是否促销日'] + [f'滞后_{lag}天' for lag in range(1, 8)]
    X = df_category[feature_columns]
    y = df_category['销量']

    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 初始化随机森林模型
    rf_model = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=42)

    # 训练模型
    rf_model.fit(X_train, y_train)

    # 使用测试集进行预测
    y_pred = rf_model.predict(X_test)

    # 过滤掉真实值为 0 的数据点，避免 MAPE 计算时除以 0
    non_zero_indices = y_test != 0
    y_test_filtered = y_test[non_zero_indices]
    y_pred_filtered = y_pred[non_zero_indices]

    # 检查是否有有效的数据点进行评估
    if len(y_test_filtered) == 0:
        print(f"品类 {category} 在测试集中所有真实销量为 0，无法计算 MAPE，跳过评估。")
        continue

    # 评估模型表现 (MAPE)
    mape = np.mean(np.abs((y_test_filtered - y_pred_filtered) / y_test_filtered)) * 100

    # 保存评估结果
    results.append({'品类': category, 'MAPE': mape})

# 转换结果为 DataFrame
results_df = pd.DataFrame(results)




# 保存评估结果到文件
results_file_path = "品类销量预测评估结果.csv"
results_df.to_csv(results_file_path, index=False)

print(f"模型评估完成，MAPE结果已保存至 {results_file_path}")


In [ ]:
import pandas as pd
import numpy as np
import joblib
import os



# 未来预测的日期范围
future_dates = pd.date_range(start="2023-07-01", end="2023-09-30")

# 获取所有品类
all_categories = [f.replace("model_", "").replace(".pkl", "") for f in os.listdir("trained_models") if f.endswith(".pkl")]

# 初始化用于保存预测结果的列表
prediction_results = []

# 遍历所有品类进行预测
for category in all_categories:
    model_path = f"trained_models/model_{category}.pkl"
    rf_model = joblib.load(model_path)

    # 创建一个 DataFrame 用于存储未来日期的特征
    df_future = pd.DataFrame({'日期': future_dates})
    df_future['月'] = df_future['日期'].dt.month
    df_future['日'] = df_future['日期'].dt.day
    df_future['星期几'] = df_future['日期'].apply(lambda date: ((5 - 1) + (date - pd.Timestamp('2022-07-01')).days) % 7 + 1)
    df_future['是否周末'] = df_future['星期几'].apply(lambda x: 1 if x in [6, 7] else 0)
    df_future['是否节假日'] = df_future['日期'].apply(lambda date: 1 if date in [
        pd.Timestamp('2023-04-05'), pd.Timestamp('2023-05-01'), pd.Timestamp('2023-06-22'),
        pd.Timestamp('2023-09-29')
    ] else 0)
    df_future['是否促销日'] = df_future['日期'].apply(lambda date: 1 if date in [
        pd.Timestamp('2023-06-18'), pd.Timestamp('2023-09-09')
    ] else 0)

    # 使用最后的已知历史数据作为起始滞后特征
    df_category = pd.read_csv("销量预测特征表.csv")
    df_category = df_category[df_category['品类'] == category]
    last_known_data = df_category.iloc[-7:]['销量'].values.tolist()

    for current_date in future_dates:
        # 构造当前日期的特征
        current_features = df_future[df_future['日期'] == current_date][['月', '日', '星期几', '是否周末', '是否节假日', '是否促销日']].copy()

        # 添加滞后特征
        for lag in range(1, 8):
            if len(last_known_data) >= lag:
                current_features[f'滞后_{lag}天'] = last_known_data[-lag]
            else:
                current_features[f'滞后_{lag}天'] = np.nan  # 如果数据不足，可以考虑填充均值等

        # 预测当前日期的销量
        current_prediction = rf_model.predict(current_features)[0]

        # 更新预测结果
        prediction_results.append({'品类': category, '日期': current_date, '预测销量': current_prediction})

        # 更新已知数据，用于下一个日期的滞后特征
        last_known_data.append(current_prediction)

        # 只保留最近 7 天的数据
        if len(last_known_data) > 7:
            last_known_data.pop(0)

            
            
# 转换预测结果为 DataFrame
prediction_results_df = pd.DataFrame(prediction_results)

# 保存预测结果到文件
prediction_results_file_path = "品类销量预测结果_逐步预测.csv"
prediction_results_df.to_csv(prediction_results_file_path, index=False)

print(f"未来销量预测完成，结果已保存至 {prediction_results_file_path}")
	

In [1]:
# 随机挑选三个品类用于可视化测试集的预测效果
selected_categories_train = random.sample(list(all_categories), min(3, len(all_categories)))
# 可视化随机挑选的品类的测试集预测效果
for category in selected_categories_train:
    # 提取当前品类的测试集真实值和预测值
    df_category = df_features[df_features['品类'] == category]
    X_train, X_test, y_train, y_test = train_test_split(
        df_category[feature_columns], df_category['销量'], test_size=0.2, random_state=42
    )
    model_path = f"trained_models/model_{category}.pkl"
    rf_model = joblib.load(model_path)
    y_pred = rf_model.predict(X_test)

    # 绘制测试集的真实值和预测值
    plt.figure(figsize=(10, 5))
    plt.plot(y_test.values, marker='o', linestyle='-', color='g', label='真实销量')
    plt.plot(y_pred, marker='x', linestyle='--', color='r', label='预测销量')
    plt.title(f'{category} 品类的测试集预测效果')
    plt.xlabel('样本索引')
    plt.ylabel('销量')
    plt.legend()
    plt.tight_layout()
    plt.show()

NameError: name 'random' is not defined

In [4]:
# 随机挑选三个品类用于可视化
import random
import pandas as pd

feature_file_path = "销量预测特征表.csv"  # 请确保文件路径正确
df_features = pd.read_csv(feature_file_path)

# 获取所有品类
all_categories = df_features['品类'].unique()

selected_categories = random.sample(list(all_categories), min(3, len(all_categories)))

# 可视化随机挑选的品类的预测结果
for category in selected_categories:
    # 提取当前品类的预测结果
    df_category_predictions = prediction_results_df[prediction_results_df['品类'] == category]
    future_dates = df_category_predictions['日期']
    category_predictions = df_category_predictions['预测销量']

    # 绘制预测结果
    plt.figure(figsize=(10, 5))
    plt.plot(future_dates, category_predictions, marker='o', linestyle='-', color='b', label='预测销量')
    
     #color自己改颜色
        
    plt.title(f'{category} 品类的销量预测 (2023年7月-9月)')
    plt.xlabel('日期')
    plt.ylabel('预测销量')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()


NameError: name 'prediction_results_df' is not defined

In [ ]:
import pandas as pd

# 读取库存量预测结果
inventory_predictions_file_path = "库存量预测结果.csv"
df_inventory = pd.read_csv(inventory_predictions_file_path, index_col=0)

# 将库存量预测结果转换为题目要求的格式（转置操作）
inventory_reshaped = df_inventory.T.reset_index()
inventory_reshaped.columns = ['品类', '7月库存量', '8月库存量', '9月库存量']

# 保存转换后的库存量预测结果
inventory_reshaped_file_path = "题目要求形式_库存量预测结果.csv"
inventory_reshaped.to_csv(inventory_reshaped_file_path, index=False)

print(f"库存量预测结果已转换并保存至 {inventory_reshaped_file_path}")

# 读取销量预测结果
sales_predictions_file_path = "品类销量预测结果_逐步预测.csv"
df_sales = pd.read_csv(sales_predictions_file_path)

# 将销量预测结果转换为题目要求的格式
sales_pivot = df_sales.pivot(index='品类', columns='日期', values='预测销量')

# 重新设置列名，将日期格式转换为 "7月1日" 这样的格式
sales_pivot.columns = [f"{pd.to_datetime(date).month}月{pd.to_datetime(date).day}日" for date in sales_pivot.columns]

# 保存转换后的销量预测结果
sales_reshaped_file_path = "题目要求形式_销量预测结果.csv"
sales_pivot.to_csv(sales_reshaped_file_path, index=True)

print(f"销量预测结果已转换并保存至 {sales_reshaped_file_path}")


In [ ]:
import pandas as pd

# 读取完整的库存量预测结果
inventory_predictions_file_path = "题目要求形式_库存量预测结果.csv"
df_inventory = pd.read_csv(inventory_predictions_file_path)

# 提取特定的品类
selected_categories = ['category1', 'category31', 'category61', 'category91', 'category121',
                       'category151', 'category181', 'category211', 'category241', 'category271',
                       'category301', 'category331']

# 提取这些品类的数据
filtered_inventory = df_inventory[df_inventory['品类'].isin(selected_categories)]

# 保存提取后的月库存量预测结果
filtered_inventory_file_path = "月库存量预测结果_提取.csv"
filtered_inventory.to_csv(filtered_inventory_file_path, index=False)

print(f"特定品类的月库存量预测结果已保存至 {filtered_inventory_file_path}")

# 读取完整的销量预测结果
sales_predictions_file_path = "题目要求形式_销量预测结果.csv"
df_sales = pd.read_csv(sales_predictions_file_path)

# 提取特定日期
selected_dates = ['7月1日', '7月11日', '7月21日', '7月31日',
                  '8月1日', '8月11日', '8月21日', '8月31日',
                  '9月1日', '9月11日', '9月21日']

# 提取这些品类和特定日期的数据
filtered_sales = df_sales[df_sales['品类'].isin(selected_categories)][['品类'] + selected_dates]

# 保存提取后的日销量预测结果
filtered_sales_file_path = "日销量预测结果_提取.csv"
filtered_sales.to_csv(filtered_sales_file_path, index=False)

print(f"特定品类的日销量预测结果已保存至 {filtered_sales_file_path}")


In [ ]:
# 可视化库存量预测结果
# 随机选择三个品类
random_categories = random.sample(selected_categories, 3)

# 读取历史库存量数据
historical_inventory_file_path = "附件1处理后.csv"
df_historical = pd.read_csv(historical_inventory_file_path, encoding='gbk')

# 过滤历史数据和预测数据
historical_data = df_historical[df_historical['品类'].isin(random_categories)]
predicted_data = filtered_inventory[filtered_inventory['品类'].isin(random_categories)]

# 颜色列表，定义每条线的颜色（可以自行调整颜色）
colors = ['#1f77b4', 'red', '#2ca02c']

# 绘制库存量历史数据和预测数据
plt.figure(figsize=(10, 6))
for i, category in enumerate(random_categories):
    # 绘制历史数据
    category_history = historical_data[historical_data['品类'] == category]
    plt.plot(category_history['月份'], category_history['库存量'], label=f"历史 - {category}", linestyle='--', color=colors[i])
    
    # 绘制预测数据
    category_prediction = predicted_data[predicted_data['品类'] == category]
    plt.plot(['7月', '8月', '9月'], category_prediction[['7月库存量', '8月库存量', '9月库存量']].values[0], 
             label=f"预测 - {category}", color=colors[i], marker='o')

plt.xlabel('月份')
plt.ylabel('库存量')
plt.title('库存量历史数据与预测数据对比')
plt.legend()
plt.grid(True)
plt.savefig('库存量预测可视化.png')
plt.show()

print("库存量预测结果可视化已保存为 '库存量预测可视化.png'")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import random

# 读取库存量数据
inventory_file_path = "附件1处理后.csv"  # 请确保文件路径正确
df_inventory = pd.read_csv(inventory_file_path, encoding='gbk')


# 转换月份列为 pandas 日期类型
df_inventory['月份'] = pd.to_datetime(df_inventory['月份'], format='%Y年%m月')

# 使用 2022 年 7 月到 2023 年 6 月的数据作为训练集
training_data = df_inventory[(df_inventory['月份'] >= '2022-07-01') & (df_inventory['月份'] <= '2023-06-01')]

# 获取所有品类
all_categories = df_inventory['品类'].unique()

# 初始化结果字典
inventory_predictions = {}

# 遍历所有品类进行预测
for idx, selected_category in enumerate(all_categories):
    try:
        # 获取该品类的数据
        data = training_data[training_data['品类'] == selected_category]['库存量'].values

        # 如果数据量不足，跳过该品类
        if len(data) < 4:  # 至少需要 4 个数据点（3 个月的输入 + 1 个月的输出）
            print(f"跳过品类 {selected_category}: 数据量不足")
            continue

        data = data.reshape(-1, 1)

        # 标准化数据
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(data)

        # 构造 LSTM 的输入序列
        sequence_length = 3  # 使用前 3 个月的数据来预测下一个月
        X_train, y_train = [], []
        for i in range(sequence_length, len(scaled_data)):
            X_train.append(scaled_data[i-sequence_length:i, 0])
            y_train.append(scaled_data[i, 0])

        X_train, y_train = np.array(X_train), np.array(y_train)

        # 调整输入数据的形状为 LSTM 所需的格式 (samples, time_steps, features)
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

        # 构建 LSTM 模型
        model = Sequential()
        model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
        model.add(LSTM(units=50))
        model.add(Dense(units=1))

        model.compile(optimizer='adam', loss='mean_squared_error')

        # 训练模型
        model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=0)

        # 准备测试数据用于预测 2023 年 7 月到 9 月的库存量
        # 使用最近的 3 个月的数据进行预测
        last_3_months = scaled_data[-sequence_length:]
        X_test = last_3_months.reshape(1, sequence_length, 1)

        # 预测未来 3 个月的库存量
        predictions = []
        for i in range(3):
            pred = model.predict(X_test)[0, 0]
            predictions.append(pred)
            # 更新输入，模拟滚动预测
            X_test = np.append(X_test[:, 1:, :], [[[pred]]], axis=1)

        # 反标准化预测结果
        predicted_stock = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

        # 保存预测结果
        inventory_predictions[selected_category] = predicted_stock.flatten()

        # 输出进度
        print(f"已完成 {idx + 1}/{len(all_categories)} 个品类的预测")

    except Exception as e:
        print(f"品类 {selected_category} 预测失败: {e}")

# 转换预测结果为 DataFrame
inventory_predictions_df = pd.DataFrame(inventory_predictions, index=['2023年7月', '2023年8月', '2023年9月']).T
inventory_predictions_df.columns = ['7月库存量', '8月库存量', '9月库存量']



# 保存预测结果到文件
inventory_predictions_file_path = "所有品类库存量预测结果.csv"
inventory_predictions_df.to_csv(inventory_predictions_file_path, index=True)

print(f"所有品类的库存量预测已完成，结果已保存至 {inventory_predictions_file_path}")


In [ ]:
# 可视化部分
# 从题目要求展示的特定品类中随机选择三个品类
selected_categories = ['category1', 'category31', 'category61', 'category91', 'category121',
                       'category151', 'category181', 'category211', 'category241', 'category271',
                       'category301', 'category331']
random_categories = random.sample(selected_categories, 3)

plt.figure(figsize=(12, 8))
for category in random_categories:
    # 获取历史数据
    category_history = training_data[training_data['品类'] == category]
    history_months = category_history['月份']
    history_values = category_history['库存量']
    
    # 获取预测数据
    future_months = pd.to_datetime(['2023-07-01', '2023-08-01', '2023-09-01'])
    future_values = inventory_predictions_df.loc[category].values
    
    # 绘制历史数据
    plt.plot(history_months, history_values, label=f'{category} 历史库存量', linestyle='--')
    
    # 绘制预测数据
    plt.plot(future_months, future_values, label=f'{category} 预测库存量', marker='o')

plt.xlabel('月份')
plt.ylabel('库存量')
plt.title('选定品类的历史库存量与预测库存量对比')
plt.legend()
plt.grid(True)
plt.savefig('选定品类库存量预测可视化.png')
plt.show()

print("选定品类的库存量预测可视化已保存为 '选定品类库存量预测可视化.png'")

In [ ]:
# 提取题目要求的特定品类的预测结果
selected_categories = ['category1', 'category31', 'category61', 'category91', 'category121',
                       'category151', 'category181', 'category211', 'category241', 'category271',
                       'category301', 'category331']
filtered_predictions_df = inventory_predictions_df.loc[selected_categories]

# 保存提取后的预测结果到文件
filtered_predictions_file_path = "题目要求品类库存量预测结果.csv"
filtered_predictions_df.to_csv(filtered_predictions_file_path, index=True)

print(f"题目要求品类的库存量预测结果已保存至 {filtered_predictions_file_path}")

In [ ]:
# 导入所需的库
import pandas as pd

# 文件路径
inventory_forecast_path = '所有品类库存量预测结果.csv'
sales_forecast_path = '题目要求形式_销量预测结果.csv'

# 读取库存量预测结果和销量预测结果
inventory_forecast_df = pd.read_csv(inventory_forecast_path, encoding='gbk')
sales_forecast_df = pd.read_csv(sales_forecast_path)

# 首先找出库存量预测数据中的品类列表和销量预测数据中的品类列表
inventory_categories = inventory_forecast_df['Unnamed: 0'].tolist()
sales_categories = sales_forecast_df['品类'].tolist()

# 找出哪些品类在销量预测数据中缺失
missing_categories = list(set(inventory_categories) - set(sales_categories))

# 为这些缺失的品类创建日销量为0的数据
# 获取已有销量预测数据的日期列，以便生成相同结构的DataFrame
dates_columns = sales_forecast_df.columns[1:]

# 为缺失的品类创建日销量为0的数据
missing_data = pd.DataFrame(0, index=missing_categories, columns=dates_columns)
missing_data.insert(0, '品类', missing_categories)

# 将原来的销量预测数据和新补全的缺失数据合并
final_sales_forecast_df = pd.concat([sales_forecast_df, missing_data], ignore_index=True)

# 按照品类名进行排序
final_sales_forecast_df = final_sales_forecast_df.sort_values(by='品类').reset_index(drop=True)

# 保存最终结果为 CSV 文件
final_sales_forecast_path = '最终日销量预测结果.csv'
final_sales_forecast_df.to_csv(final_sales_forecast_path, index=False)

print("最终的日销量预测结果表格已成功保存到：", final_sales_forecast_path)
